# Trackpad Data Exploration

This notebook provides interactive exploration of trackpad session data collected from the savantlab macOS app.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import analysis modules
import data_loader
import metrics
import visualize

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Session Data

First, specify the directory containing your session CSV files.

In [ ]:
# Update this path to your session data directory
SESSION_DIR = Path.home() / 'Library/Containers/savant.savantlab/Data/Documents/savantlab-trackpad-sessions'

# Load all sessions
sessions = data_loader.load_sessions_from_directory(SESSION_DIR)

print(f"Loaded {len(sessions)} sessions")
for session in sessions[:5]:  # Show first 5
    print(f"  {session.session_id}: {len(session.df)} events")

## 2. Explore a Single Session

Let's examine one session in detail.

In [ ]:
# Select a session (change index to explore different sessions)
session = sessions[0]

print(f"Session: {session.session_id}")
print(f"Start time: {session.start_time}")
print(f"\nSummary:")
summary = session.summary()
for key, value in summary.items():
    print(f"  {key}: {value}")

In [ ]:
# View raw data
session.df.head(10)

## 3. Compute Metrics

Calculate velocity, acceleration, and distance traveled.

In [ ]:
# Compute all metrics for pointer events
metrics_df = metrics.compute_all_metrics(session, 'pointer')

print(f"Computed metrics for {len(metrics_df)} pointer events")
metrics_df.head()

In [ ]:
# Show statistics
stats = metrics.session_statistics(metrics_df)
for key, value in stats.items():
    print(f"{key}: {value}")

## 4. Visualize Session Data

Create various plots to explore the data.

In [ ]:
# Comprehensive overview
fig = visualize.plot_session_overview(session, metrics_df)
plt.show()

In [ ]:
# Trajectory colored by speed
visualize.plot_trajectory(metrics_df, color_by='speed', title='Trajectory (colored by speed)')
plt.show()

In [ ]:
# Speed distribution
visualize.plot_speed_distribution(metrics_df)
plt.show()

In [ ]:
# Movement direction (polar plot)
visualize.plot_direction_polar(metrics_df)
plt.show()

## 5. Compare Multiple Sessions

Analyze trends across sessions.

In [ ]:
# Compare sessions (if you have multiple with data)
if len(sessions) > 1:
    comparison_df = metrics.compare_sessions(sessions, 'pointer')
    print("Session comparison:")
    display(comparison_df)
else:
    print("Need at least 2 sessions for comparison")

In [ ]:
# Plot comparison for a specific metric
if len(sessions) > 1:
    visualize.plot_session_comparison(sessions, metric='total_distance')
    plt.show()

## 6. Time-Binned Analysis

Look at how metrics change over time within a session.

In [ ]:
# Compute time-binned statistics (1 second bins)
binned = metrics.time_binned_statistics(metrics_df, bin_size_sec=1.0)

if not binned.empty:
    print("Time-binned statistics:")
    display(binned.head(10))
else:
    print("No data for time binning")

## 7. Export Processed Data

Save metrics and statistics for further analysis.

In [ ]:
# Export metrics to CSV
output_dir = Path('output')
output_dir.mkdir(exist_ok=True)

metrics_df.to_csv(output_dir / f'{session.session_id}_metrics.csv', index=False)
print(f"Exported metrics to {output_dir / f'{session.session_id}_metrics.csv'}")

In [ ]:
# Export session statistics as JSON
import json

with open(output_dir / f'{session.session_id}_stats.json', 'w') as f:
    json.dump(stats, f, indent=2, default=str)

print(f"Exported statistics to {output_dir / f'{session.session_id}_stats.json'}")

## 8. Custom Analysis

Use this section for your own exploratory analysis.

In [ ]:
# Your custom analysis here
